In [1]:
# Load Packages
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import requests
import re

**Extracting Goals and Players who scored - Done**

In [2]:
# Define URL to scrap
url = "https://rfef.es/es/competiciones/division-de-honor-juvenil/acta/3886158?group=249&round=65034"

In [3]:
# Send a GET request to the URL and get the HTML response
response = requests.get(url)

In [4]:
# Create a BeautifulSoup object from the HTML content
#soup = BeautifulSoup(response.content, "html.parser")
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
def extract_starting11_goals(soup, table_name):
    table = soup.find("table", class_=table_name)
    
    team1 = []
    team2 = []
    for count, team  in enumerate(table.find_all("ul")):
        for player in team.find_all("li"):
            if count == 0:
                team1.append(player.text.strip())
            else:
                team2.append(player.text.strip())
    return team1, team2

In [6]:
def extract_minutes(team1, team2):
    minutes1 = map(lambda x: x[:-1].split(" ")[1], team1)
    minutes2 = map(lambda x: x[:-1].split(" ")[1], team2)
    
    team1 = map(lambda x: x[:-3], team1)
    team2 = map(lambda x: x[:-3], team2)
    
    return list(minutes1), list(minutes2), list(team1), list(team2)    

In [7]:
team1, team2 = extract_starting11_goals(soup, "acta-result-jugardores-table")
minutes1, minutes2, team1, team2 = extract_minutes(team1, team2)

In [8]:
pd.DataFrame({"team1":pd.Series(team1), 
              "minutes1":pd.Series(minutes1), 
              "team2":pd.Series(team2), 
              "minutes2":pd.Series(minutes2)})

,team1,minutes1,team2,minutes2
0,Pablo,53,Vivas 76' (,76'
1,Christian,61,NaN,NaN
2,Ivan,89,NaN,NaN


**Figure Out a Way to Extract URL programatically**

In [9]:
# function to extract html document from given url
def getHTMLdocument(url):
      
    # request for HTML document of given url
    response = requests.get(url)
      
    # response will be provided in JSON format
    return response.text
  
    
# assign required credentials
# assign URL
url_to_scrape = "https://rfef.es/es/competiciones/division-de-honor-juvenil/"
  
# create document
html_document = getHTMLdocument(url_to_scrape)
  
# create soap object
soup = BeautifulSoup(html_document, 'html.parser')
  
  
# find all the anchor tags with "href" 
# attribute starting with "https://"
for link in soup.find_all('a', 
                          attrs={'href': re.compile("^https://")}):
    # display the actual urls
    print(link.get('href'))  

https://rfef.es/es
https://es.fifa.com/who-we-are/official-documents/
https://www.rfef.es/sites/default/files/pdf/codigo-de-buen-gobierno.pdf
https://tickets.rfef.es/
https://shop.rfef.es/
https://fenix.rfef.es/GestorPeticiones/
https://fenix.rfef.es/sso/
https://rfef.es/es/noticias/gonzalo-vale-una-copa-de-campeones
https://rfef.es/es/noticias/alvaro-arbeloa-con-este-titulo-cerramos-un-triplete-historico-y-acabamos-invictos
https://rfef.es/es/noticias/la-celebracion-del-real-madrid-cf-tras-ganar-la-copa-de-campeones
https://rfef.es/es/noticias/el-real-madrid-cf-gana-al-real-betis-balompie-1-3-y-se-hace-con-su-octavo-titulo
https://rfef.es/es/noticias/juveniles-futbol-11/copa-de-campeones
https://www.youtube.com/watch?v=TMy-eMJqjN4
https://www.youtube.com/watch?v=m2snltJmPAY
https://rfef.es/es/noticias/gonzalo-vale-una-copa-de-campeones
https://rfef.es/es/noticias/gonzalo-vale-una-copa-de-campeones
https://rfef.es/es/noticias/alvaro-arbeloa-con-este-titulo-cerramos-un-triplete-historic

In [10]:
url = "https://rfef.es/es/competiciones/division-de-honor-juvenil/"#"https://rfef.es/es/competiciones/division-de-honor-juvenil/"

response = requests.get(url)
html_content = response.text

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

In [11]:
# Find all the links in the HTML content using BeautifulSoup
links = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href is not None:
        links.append(href)

In [12]:
# Find all the URLs in the HTML content using regular expressions
urls = re.findall(r'/es/competiciones/division-de-honor-juvenil', html_content)

# Print the links and URLs
#print('Links:', links)
print('URLs:', urls)


URLs: ['/es/competiciones/division-de-honor-juvenil', '/es/competiciones/division-de-honor-juvenil', '/es/competiciones/division-de-honor-juvenil', '/es/competiciones/division-de-honor-juvenil', '/es/competiciones/division-de-honor-juvenil', '/es/competiciones/division-de-honor-juvenil', '/es/competiciones/division-de-honor-juvenil']


In [ ]:
dom = etree.HTML(str(soup))
print(dom.xpath('//*[@id="resultados-tab"]/span')[0].text)

In [ ]:
url = "https://rfef.es/es/competiciones/division-de-honor-juvenil/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html5lib")

In [ ]:
print(soup.prettify())

In [15]:
<a class="we-mega-menu-li" href="http://actas.rfef.es/actas/RFEF_CmpJornada?cod_primaria=1000144&amp;CodCategoria=100" target="" title="">
                            Actas

SyntaxError: invalid syntax (1059753293.py, line 1)

**Table for starting 11 - WORK IN PROGRESS**

In [13]:
table = soup.find("table", class_="acta-titulares-table")
row = table.find_all("tr")
team1, minute1 = [], []
team2, minute2 = [], []
for i in range(1, len(row)):
    players = row[i].find_all("td")
    player1, player2 = str(players[0].text.strip()), str(players[1].text.strip())
    player1, player2 = re.sub("^[1-9]|'| ","",player1).split("\n"), re.sub("^[1-9]|'| ","",player2).split("\n")
    while("" in player1):
        player1.remove("")
    try:
        team1.append(player1[0])
        minute1.append(player1[1])
    except IndexError:
        minute1.append(str(""))
    while("" in player2):
        player2.remove("")
    try:
        team2.append(player2[0])
        minute2.append(player2[1])
    except IndexError:
        minute2.append(str(""))

AttributeError: 'NoneType' object has no attribute 'find_all'

In [14]:
minute2

NameError: name 'minute2' is not defined